### **TD 11 et 12 - Collecte, Traitement, et Analyse de données de réseaux sociaux**

*LEFEVRE Laura et LE CORRE Camille - LDD BI*

In [27]:
#Importation des modules
import json
import re
import pandas as pd
from textblob import TextBlob
#import textblob_fr

#Module a installer
    #!pip install textblob
    #!pip install textblob-fr

In [9]:
def openFile(fileName):
    with open(fileName, encoding='utf8') as json_data:
        data_dict = json.load(json_data)
        return data_dict

In [15]:
dic = openFile("versailles_tweets_100.json")
dic[2]


{'_id': '1421599163561742339',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 1,
  'like_count': 1,
  'quote_count': 0},
 'id': '1421599163561742339',
 'conversation_id': '1421599163561742339',
 'context_annotations': [{'domain': {'id': '6', 'name': 'Sports Event'},
   'entity': {'id': '1103026158382141440',
    'name': 'Tokyo 2020 Summer Olympics',
    'description': 'Tokyo 2020 Summer Olympics '}}],
 'author_id': '1339914264522461187',
 'text': 'Ah oui le sommeil là sera compliqué. #CIV  est éliminé des JO , Ahi on peut faire ça ?',
 'geo': {'place_id': '00b8943291443c8c'},
 'lang': 'fr',
 'created_at': '2021-07-31T22:30:27.000Z',
 'entities': {'hashtags': [{'start': 37, 'end': 41, 'tag': 'CIV'}]}}

In [159]:
dic[0]['text']

'Goumin des éléphants joueurs la même fatigue même 😫 #twitter225'

In [10]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [11]:
def CleanText(text):
    text = text.lower()
    text = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", text)
    text = re.sub(r"[\,\!\?\%\(\)\/\"\&\+\#\$\£\%\:\@\-]", "", text)
    text = re.sub(r"[\é\è\ê]", "e", text)
    text = re.sub(r"[\ù]", "u", text)
    text = re.sub(r"[\à]", "a", text)
    text = re.sub(r"[\î\ï]", "i", text)
    text = remove_emojis(text)
    text = re.sub(r"( )+", " ", text)
    text = re.sub(r"( )*$", "", text)
    
    return text

In [164]:
CleanText(dic[0]['text'])

'goumin des elephants joueurs la meme fatigue meme'

In [19]:
def ZoneAtterrissage(tweet):
    tweet_clean = CleanText(tweet['text'])
    tweet['text'] = tweet_clean
    with open("zone_atterrissage.json", "w") as filout:
        json.dump(tweet, filout)
        #json.dump(tweet + '\n', filout)


In [20]:
def traitement(liste_tweet):
    
    for elt in liste_tweet:
        ZoneAtterrissage(elt)

In [21]:
traitement(dic)

In [46]:
testimonial = TextBlob("Textblob is amazingly simple to use. What great fun!")
testimonial.sentiment

Sentiment(polarity=0.39166666666666666, subjectivity=0.4357142857142857)

In [42]:
def ListHashtags(tweet):
    ''' Fonction qui renvoie la liste de hashtags d'un tweet.
    Cette fonction est utilisée dans la fonction ZoneEntrepot '''

    list_hashtags = []

    if "hashtags" in tweet["entities"]:                # on vérifie qu'il y a au moins un hashtag
        for h in range(len(tweet["entities"]["hashtags"])):            # on parcourt la liste des hashtags
            list_hashtags.append(tweet["entities"]["hashtags"][h]["tag"])   

    return list_hashtags

In [23]:
def ListMentionedUsers(tweet):
    ''' Fonction qui renvoie la liste des utilisateurs mentionnés dans un tweet.
    Cette fonction est utilisée dans la fonction ZoneEntrepot '''

    list_mentioned_users = []

    if "mentions" in tweet["entities"]:                # on vérifie qu'il y a au moins un utilisateur mentionné
        for m in range(len(tweet["entities"]["mentions"])):            # on parcourt la liste des utilisateurs mentionnés
            list_mentioned_users.append(tweet["entities"]["mentions"][m]["username"])

    return list_mentioned_users

In [25]:
def Topic(tweet):
    ''' Fonction qui détermine le topic d'un tweet.
    Cette fonction est utilisée dans la fonction ZoneEntrepot '''
    pass

In [35]:
# /!\ il faut trouver le paramètre pour l'adapter en français ou toutes les langues

def Feeling(tweet):
    ''' Fonction qui détermine le sentiment d'un tweet (positif, négatif ou neutre).
    Cette fonction est utilisée dans la fonction ZoneEntrepot '''
    
    feeling = TextBlob(str(tweet.get("texte"))).sentiment.polarity

    if feeling < 0:
        return "negative"
    elif feeling == 0:
        return "neutral"
    else:
        return "positive"

In [38]:
def ZoneEntrepot(list):
    '''Fonction qui prend en entrée la liste des dictionnaires correspondant
    aux tweets issus du fichier zone_atterrissage et qui créer un DataFrame
    contenant uniquement les informations qui nous intéressent'''

    # Création d'un DataFrame (chaque ligne va correspondre à un tweet)
    zone_entrepot = pd.DataFrame(columns = ['author_id', 'text', 'hashtags', 'mentioned_users', 'topics', 'feelings'])

    for k in range(len(list)-1):        # on travaille sur un tweet à la fois
        # Création d'une liste qui va contenir toutes les informations utiles de ce tweet
        tweet = []
        # Ajout à cette liste des informations récupérées de la zone d'atterissage
        tweet.append(list[k].get("author_id"))              # auteur du tweet    
        tweet.append(list[k].get("text"))                   # texte du tweet
        tweet.append(ListHashtags(list[k]))                 # liste des hashtags
        tweet.append(ListMentionedUsers(list[k]))           # liste des utilisateurs mentionnés
        tweet.append(Topic(list[k]))                        # topic
        tweet.append(Feeling(list[k]))                      # sentiment
        
        # Ajout de la ligne correspondant à ce tweet à notre DataFrame
        zone_entrepot.loc[k+1] = tweet

    return zone_entrepot    

In [39]:
df_zone_entrepot = ZoneEntrepot(dic)

KeyError: 'entities'

In [7]:
def ListColumn(df, column:str):
    ''' Fonction qui renvoie les éléments d'une colonne d'un DataFrame sous forme d'une liste'''

    if df[column][1] == int:            # si la colonne contient des entiers
        return list(df[column])
    elif df[column][1] == list:         # si la colonne contient des listes (une liste de hashatgs
                                        # ou d'utilisateurs mentionnés par exemple)
        l_lists = list(df[column])
        l_elem = []
        for e in l_lists:
            for ee in e:
                l_elem.append(ee)
        return l_elem

In [2]:
def NbOccurencesIntoDict(l):
    ''' Fonction qui crée un dictionnaire dans lequel chaque clé
    correspond à un utilisateur, et chaque valeur correspond au nombre
    d'occurrences de cet utilisateur dans la liste'''

    dic = {}

    for elem in l:
        if elem in dic:
            dic[elem] += 1
        else:
            dic[elem] = 1

    return dic

In [3]:
def TopKUsers(df, k):
    ''' Fonction renvoyant les k utilisateurs ayant publié le plus de tweets'''

    l_users = ListColumn(df, "author_id")
    dic_occurences = NbOccurencesIntoDict(l_users)
    #dic_occurences = NbOccurencesIntoDict(ListColumn(df, "author_id"))

    l_sorted = list(sorted(dic_occurences, key=dic_occurences.get, reverse=True))

    return l_sorted[:k]